In [ ]:
!pip install opendatasets --upgrade --quiet
import opendatasets as od
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import ImageFolder
from kaggle.api.kaggle_api_extended import KaggleApi
dataset_url='https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia'
od.download(dataset_url)


In [ ]:
dataset_directory = '/content/chest-xray-pneumonia'
os.makedirs(dataset_directory, exist_ok=True)

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])
class ChestDataset(Dataset):
    def __init__(self, data_path, transform=None):
        self.dataset = ImageFolder(root=data_path, transform=transform)

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        image, label = self.dataset[index]
        return image, label

train_data_path = os.path.join(dataset_directory, 'chest_xray/train')
val_data_path = os.path.join(dataset_directory, 'chest_xray/val')
test_data_path = os.path.join(dataset_directory, 'chest_xray/test')

train_dataset = ChestDataset(train_data_path, transform=transform)
val_dataset = ChestDataset(val_data_path, transform=transform)
test_dataset = ChestDataset(test_data_path, transform=transform)

batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

class ChestCNN(nn.Module):
    def __init__(self):
        super(ChestCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(16 * 128 * 128, 64)  # Adjust the input size depending on your image size
        self.fc2 = nn.Linear(64, 2)  # 2 output classes: Normal and Pneumonia

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = x.view(-1, 16 * 128 * 128)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = ChestCNN()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Step 11: Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f"Epoch [{epoch + 1}/{num_epochs}] - Loss: {loss.item():.4f}")

# Step 12: Evaluation on validation set
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in val_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Validation Accuracy: {accuracy:.2f}%")

# Step 13: Evaluation on test set
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")